<a href="https://colab.research.google.com/github/AbrahamKong/CMPE256-Vector_Space_Assignment/blob/main/CMPE_256_Vector_Space_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import neccassary library

import pandas as pd
import numpy as np

In [ ]:
# Read In text from files

data_dir = "/content/"
data_files = [
    "d1.txt", "d2.txt", "d3.txt",
    "d4.txt"
]

data = pd.Series()
for file in data_files:
    f = open(data_dir + file, 'r')
    file_contents = f.read()
    data[file[:-4]] = file_contents
    f.close()
    print(data[file[:-4]])

User selected Wedding gown.
User ordered on-line rose flowers.
User searched diamond ring.
User selected white wedding gown, online flowers, 3 carat diamond ring.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [ ]:
# Count Word for each document

from collections import Counter

bag_of_words = (
    data.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)

bag_of_words

d1    {'user': 1, 'selected': 1, 'wedding': 1, 'gown...
d2    {'user': 1, 'ordered': 1, 'on': 1, 'line': 1, ...
d3    {'user': 1, 'searched': 1, 'diamond': 1, 'ring...
d4    {'user': 1, 'selected': 1, 'white': 1, 'weddin...
dtype: object

**TF-iDF**

In [ ]:
# Set up TD

tf = pd.DataFrame(list(bag_of_words))
tf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
# Fill N/A value with 0

tf = tf.fillna(0)
tf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
# Get document frequencies 

df = (tf > 0).sum(axis=0)
df

user        4
selected    2
wedding     2
gown        2
ordered     1
on          1
line        1
rose        1
flowers     2
searched    1
diamond     2
ring        2
white       1
online      1
3           1
carat       1
dtype: int64

In [ ]:
# Get IDFs


idf = np.log(len(tf) / df)
idf

user        0.000000
selected    0.693147
wedding     0.693147
gown        0.693147
ordered     1.386294
on          1.386294
line        1.386294
rose        1.386294
flowers     0.693147
searched    1.386294
diamond     0.693147
ring        0.693147
white       1.386294
online      1.386294
3           1.386294
carat       1.386294
dtype: float64

In [ ]:
# Calculate TF-IDFs


tf_idf = tf * idf
tf_idf

,user,selected,wedding,gown,ordered,on,line,rose,flowers,searched,diamond,ring,white,online,3,carat
0,0.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,1.386294,1.386294,1.386294,1.386294,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000
3,0.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.693147,0.693147,1.386294,1.386294,1.386294,1.386294


**Cisine Similarity**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(data) # This determines the vocabulary.
tf_idf_sparse = vec.transform(data)

In [ ]:
# Calculate the numerator.
a = tf_idf_sparse[0, :]
b = tf_idf_sparse[2, :]
dot = a.multiply(b).sum()

# Calculate the terms in the denominator.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(b.multiply(b).sum())

# Cosine similarity is their ratio.
dot / (a_len * b_len)

0.11744989547797277

In [ ]:
# Calculate the numerators.
a = tf_idf_sparse[0, :]
B = tf_idf_sparse
dot = a.multiply(B).sum(axis=1)
dot

matrix([[7.8477822],
        [1.       ],
        [1.       ],
        [7.8477822]])

In [ ]:
# Calculate the denominators.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(B.multiply(B).sum(axis=1))
print(a_len)
b_len

2.8013893332208726


matrix([[2.80138933],
        [4.23925403],
        [3.03930227],
        [5.07070753]])

In [ ]:
# Calculate their ratio to obtain cosine similarities.
dot / (a_len * b_len)

matrix([[1.        ],
        [0.08420485],
        [0.1174499 ],
        [0.55246518]])

In [ ]:
cos_similarities = pd.DataFrame(dot / (a_len * b_len))[0]
most_similar = cos_similarities.sort_values(ascending=False)
most_similar

0    1.000000
3    0.552465
2    0.117450
1    0.084205
Name: 0, dtype: float64